In [1]:
import src.pmcmc.main as pmcmc_main

In [2]:
pmcmc_main.main("06", "2024-04-27")

PMCMC Progress: 0it [00:00, ?it/s]                                        1.51s/it]


Array([[0.001     , 0.0160432 , 0.01137184, ..., 0.10509229, 0.10826205,
        0.10643896],
       [0.03911028, 0.02117727, 0.01048695, ..., 0.11249776, 0.10198469,
        0.10087529],
       [0.03890042, 0.00979877, 0.01719936, ..., 0.13215451, 0.13641371,
        0.09556162],
       ...,
       [0.0077249 , 0.03675001, 0.01247766, ..., 0.1325028 , 0.08701457,
        0.11388317],
       [0.01654181, 0.02720235, 0.02271833, ..., 0.128433  , 0.09967344,
        0.09053642],
       [0.0024256 , 0.03214093, 0.02477897, ..., 0.10575061, 0.13215952,
        0.09792652]], dtype=float32)

In [3]:
pmcmc_main.main("04", "2024-04-27")

PMCMC Progress: 0it [00:00, ?it/s]                                        1.49s/it]


Array([[0.001     , 0.01413135, 0.00945999, ..., 0.11513677, 0.11830653,
        0.11648345],
       [0.03911028, 0.01926542, 0.01370917, ..., 0.11888187, 0.11202918,
        0.12203167],
       [0.03890042, 0.00788692, 0.00390901, ..., 0.12524697, 0.12314107,
        0.12669365],
       ...,
       [0.0077249 , 0.03675001, 0.01577767, ..., 0.08991994, 0.08591162,
        0.10657184],
       [0.01654181, 0.02720235, 0.03227675, ..., 0.09493215, 0.09701283,
        0.08301048],
       [0.0024256 , 0.03214093, 0.03067963, ..., 0.10973269, 0.10938941,
        0.10764249]], dtype=float32)

In [4]:
import os
import numpy as np
import pandas as pd
from src.utils import paths

locations = ["04", "06"]
target_dates = pd.read_csv("../datasets/target_dates.csv")

In [7]:
dates = target_dates["date"].values

In [8]:
for loc in locations:
    print("Working location ", loc)
    base_dir = os.path.join(paths.OUTPUT_DIR, "pmcmc_runs")
    loc_dir = os.path.join(base_dir, loc)
    beta_file_path = os.path.join(loc_dir, "mle_betas_20241020.csv")
    df = pd.read_csv(beta_file_path)

    # Average over the particles
    row_means = df.mean(axis=0)
    # Transpose the DataFrame
    transposed_df = row_means.to_frame()
    num_rows = len(transposed_df)
    end_date = pd.to_datetime("2024-04-27")
    date_range = pd.date_range(end=end_date, periods=num_rows)
    transposed_df["date"] = date_range
    transposed_df.rename({0: "beta"}, axis="columns", inplace=True)

    df = transposed_df
    for target_date in target_dates["date"]:
        start_date = pd.to_datetime(target_date) - pd.Timedelta(days=50)
        filtered_df = df[(df["date"] > start_date) & (df["date"] <= target_date)]
        filtered_df.insert(0, "time_0", range(len(filtered_df)))
        filtered_df = filtered_df.drop(columns=["date"])
        file_path = os.path.join(paths.PF_OUTPUT_DIR, target_date, f"{loc}.csv")
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        filtered_df.to_csv(file_path, index=False)

Working location  04
Working location  06


In [15]:
base_dir = os.path.join(paths.OUTPUT_DIR, "pmcmc_runs")
loc_dir = os.path.join(base_dir, "04")
mle_states_path = os.path.join(loc_dir, "mle_states_20241020.npy")
states_np = np.load(mle_states_path)
start_date_index = abs(
    (pd.to_datetime("2023-10-21") - pd.to_datetime("2023-06-25")).days
)
end = start_date_index + 8 * 7
new_h = states_np[:, start_date_index - 1 : end]
new_h_df = pd.DataFrame(new_h)
new_h

array([[ 5.732454 ,  6.051192 ,  6.560311 , ..., 90.70723  , 93.82167  ,
        98.93495  ],
       [ 5.7523856,  6.092282 ,  5.884512 , ..., 65.38718  , 92.88246  ,
        98.21516  ],
       [ 5.4401956,  5.6333823,  6.114605 , ..., 88.318794 , 92.074844 ,
        97.717316 ],
       ...,
       [ 5.5394583,  5.891465 ,  6.354831 , ..., 85.366684 , 91.00736  ,
        98.99488  ],
       [ 5.5303836,  5.847938 ,  6.28207  , ..., 85.366684 , 90.15415  ,
        96.389015 ],
       [ 5.235332 ,  5.4014287,  5.644407 , ..., 85.42829  , 90.15415  ,
        96.05348  ]], dtype=float32)

In [16]:
end - start_date_index

56

In [18]:
quantiles = new_h_df.quantile([0.025, 0.5, 0.975], axis=0)
quantiles

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0.025,4.699870,5.230518,5.625346,6.258797,6.885636,6.586511,6.097719,6.238450,6.264276,6.408696,...,48.152379,48.573862,50.483437,52.825526,56.223544,59.874029,64.621458,70.395978,72.895884,77.077866
0.500,6.203783,6.627595,7.241414,8.089949,8.993998,8.803933,8.527929,8.676049,8.606984,8.677137,...,58.543699,59.072258,61.297937,63.732174,67.398499,72.960464,79.288811,83.870605,86.951008,91.560188
0.975,7.489518,8.133594,8.881882,9.746368,10.674457,10.929939,11.053963,11.432455,11.652841,11.573535,...,72.448473,71.881641,75.833715,75.704381,79.285799,85.819214,94.085790,99.852579,103.255623,109.645516


In [23]:
new_h_weekly = new_h_df.groupby(new_h_df.columns // 7, axis=1).sum()

/tmp/ipykernel_1107844/98241468.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  new_h_weekly = new_h_df.groupby(new_h_df.columns // 7, axis=1).sum()


In [24]:
new_h_weekly

,0,1,2,3,4,5,6,7,8
0,46.684170,58.390144,71.493874,110.400131,175.835602,320.646149,405.397339,571.019470,98.934952
1,44.432716,58.920731,74.190720,122.674164,197.430939,318.098633,414.593201,470.964325,98.215157
2,44.747322,59.875587,74.215858,118.104126,203.473328,321.772461,384.076965,479.819275,97.717316
3,43.785320,60.556664,81.780212,119.289528,202.728729,291.377991,367.350647,486.546173,85.558662
4,45.056236,60.876068,79.241219,120.723846,186.048050,257.195923,345.814941,484.600098,87.473106
...,...,...,...,...,...,...,...,...,...
295,47.739929,66.314110,74.841583,125.086975,208.970703,312.853821,394.441498,502.668884,90.237434
296,46.854725,69.314758,74.918510,122.837936,211.457397,308.822388,396.138550,500.160858,96.681526
297,47.240307,68.140274,78.641792,119.721336,210.068497,306.746063,405.496552,529.254028,98.994881
298,46.738396,69.240433,78.775658,119.370926,209.112473,296.800873,415.294800,503.650513,96.389015


In [19]:
import os
import pandas as pd
from src.utils import paths
import numpy as np


prediction_date = pd.to_datetime("2023-11-18")
start_date = prediction_date - pd.DateOffset(weeks=8)
base_dir = os.path.join(paths.OUTPUT_DIR, "pmcmc_runs")
loc_dir = os.path.join(base_dir, "04")
mle_states_path = os.path.join(loc_dir, "mle_states_20241020.npy")
states_np = np.load(mle_states_path)
last_date_index = abs((prediction_date - pd.to_datetime("2023-06-25")).days)
start_index = last_date_index - (8 * 7)
new_h = states_np[:, start_index:last_date_index]
new_h_df = pd.DataFrame(new_h)
full_date_range = pd.date_range(start=start_date, periods=new_h_df.shape[1], freq="D")
new_h_df_T = new_h_df.T

In [20]:
new_h_df_T

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,2.340812,2.222163,2.197886,1.433856,1.457902,1.409063,1.383808,1.631343,1.619501,1.596034,...,0.960460,0.878564,0.884867,0.883102,0.807896,0.814499,0.749703,0.749048,0.728597,1.512661
1,2.438141,2.438141,2.305723,2.256570,1.431667,1.464359,1.401217,1.376131,1.376131,1.650395,...,0.923808,0.904969,1.534475,0.920608,0.844727,0.860162,0.788333,0.730479,0.728315,1.522138
2,2.554531,2.417758,2.344902,1.434372,1.474994,1.403542,1.374564,1.376362,1.692543,1.644074,...,0.890331,1.569187,0.886662,0.886662,0.810738,0.838463,0.780517,0.716350,0.713405,1.546358
3,2.652844,2.652844,2.652844,2.652844,2.533273,2.533273,2.533273,2.423476,2.423476,2.423476,...,2.318136,0.803793,2.018397,2.018397,0.837623,0.888325,1.586089,0.846934,0.697369,1.564482
4,2.794218,2.801889,2.813131,2.714159,2.672424,2.552599,2.547917,1.475097,1.498358,1.429119,...,2.413977,0.802880,2.040484,2.068822,0.841240,0.889141,1.633239,0.826029,0.694799,1.608088
5,2.890471,2.890471,2.890471,2.890471,2.930552,2.930552,2.930552,2.930552,2.930552,2.939616,...,2.476031,2.476031,2.483780,2.483780,2.483780,2.021159,2.021159,2.086903,0.877199,1.621106
6,2.966788,2.942041,2.934510,2.951087,3.028972,3.012835,3.037657,3.033703,3.035168,3.054260,...,2.497010,2.524836,2.530924,2.544371,2.543899,1.981318,1.947182,2.085140,2.085140,1.628139
7,3.061717,3.061717,3.004108,3.003960,3.003960,3.039000,3.039000,3.155303,3.133510,3.133510,...,2.579730,2.532004,2.571948,2.609286,2.609286,2.633254,2.632520,1.955451,2.093122,2.108873
8,3.176625,3.167387,3.079717,3.086337,3.083487,3.139898,3.142707,3.307283,3.264204,3.370903,...,2.561612,2.636607,2.672341,2.681801,2.749046,2.725435,2.725435,1.928794,2.125262,2.149630
9,3.285783,3.285783,3.302639,3.163738,3.189736,3.192760,3.262474,3.287762,3.287762,3.451585,...,2.674119,2.608543,2.708245,2.732085,2.781575,2.903733,2.843958,2.815049,1.912107,2.146837


In [26]:
new_h_weekly = new_h_df.groupby(new_h_df.columns // 7, axis=1).sum()
quantiles = new_h_df_T.quantile([0.025, 0.5, 0.975], axis=1)

/tmp/ipykernel_1173769/4283147346.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  new_h_weekly = new_h_df.groupby(new_h_df.columns // 7, axis=1).sum()


In [28]:
quantiles = quantiles.T

In [29]:
quantiles["date"] = full_date_range
quantiles_subset = quantiles[
    (quantiles["date"] >= start_date) & (quantiles["date"] <= prediction_date)
]
quantiles_subset = quantiles.resample("W-Sat", on="date").sum().reset_index()

In [30]:
quantiles_subset

,date,0.025,0.5,0.975
0,2023-09-23,0.675154,1.243847,2.256812
1,2023-09-30,7.066018,13.281118,20.777685
2,2023-10-07,11.277836,19.897578,28.564048
3,2023-10-14,10.005897,18.593159,29.947693
4,2023-10-21,27.401678,37.796705,46.827918
5,2023-10-28,43.956736,58.940255,74.371905
6,2023-11-04,46.944722,62.294646,83.976008
7,2023-11-11,67.363258,84.932618,96.848954
8,2023-11-18,100.675372,116.938420,130.772354
